## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [19]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200



***Connection broken [Device not configured]
You may need to reconnect
Closing serial Serial<id=0x7fe6e6ead760, open=True>(port='/dev/tty.usbserial-0001', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [41]:
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'CCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


In [44]:
a = [
                0.0008810108,
                0.6322876,
                5.302984,
                11.04289,
                7.236574,
                4.483564,
                8.169661,
                11.45558,
                12.94468,
                0.3870013,
                0.5778582,
                8.11299,
                -0.4887207,
                -1.699883,
                -1.151954,
                5.314356,
                2.863382,
                -6.663355,
                -4.236682,
                -0.3282205,
                11.8786,
                10.40834,
                2.91623,
                4.891585,
                4.118226,
                6.573776,
                7.595751,
                -0.3148028,
                -11.22296,
                -13.47364,
                -20.09728,
                -24.40007,
                -11.1898,
                0.5727892,
                0.9114911,
                -8.969307,
                -14.68794,
                -14.8013,
                -14.64045,
                -1.218613,
                -3.024044,
                -12.22603,
                8.065958,
                1.758298,
                -27.88097,
                -27.54515,
                -8.046073,
                3.199111,
                -0.7088236,
                11.4001,
                17.5606,
                0.7582185,
                -1.796498,
                -4.495961,
                -8.028107,
                13.39242,
                16.89291,
                -0.5617995,
                9.200119,
                5.101417,
                -6.820274,
                9.410563,
                18.42916,
                9.263607,
                0.002051557,
                0.5940795,
                4.680033,
                8.696577,
                0.4415902,
                -7.68333,
                -3.404639,
                -4.519523,
                -10.17533,
                -8.14777,
                2.611212,
                0.867302,
                -13.66548,
                -14.09752,
                -11.8542,
                -4.321161,
                7.699627,
                7.135615,
                4.134768,
                0.4307611,
                -10.29667,
                -16.14585,
                -5.777066,
                8.264203,
                11.61974,
                -6.38609,
                -22.92923,
                -16.96802,
                -11.00848,
                8.671667,
                24.75481,
                7.650834,
                8.493778,
                10.34663,
                -3.488946,
                10.08992,
                20.18735,
                14.38455,
                -0.2793979,
                -7.029641,
                9.924578,
                1.700456,
                -12.34403,
                -0.5314187,
                6.344598,
                -5.486998,
                -13.30798,
                3.852185,
                8.454625,
                -5.823669,
                -0.6756685,
                2.385444,
                -10.81458,
                -5.086719,
                9.13818,
                7.320928,
                1.16188,
                3.629816,
                6.896602,
                8.682119,
                9.012561,
                -2.122387,
                4.7857,
                -0.3938826,
                0.003268619,
                -0.07217627,
                -0.3948116,
                1.617305,
                6.779059,
                9.026737,
                14.23627,
                18.07122,
                11.43218,
                11.11577,
                7.619313,
                13.80112,
                24.14718,
                12.22281,
                15.89785,
                14.14789,
                10.03987,
                18.32486,
                10.04507,
                4.5887,
                5.286398,
                2.128071,
                -7.647141,
                -3.705783,
                -6.877206,
                -13.72737,
                -3.713557,
                -11.5692,
                -12.10889,
                -12.25982,
                -10.81629,
                -10.02413,
                -24.07154,
                -13.21079,
                -1.02243,
                0.8013479,
                -4.524598,
                -19.12002,
                -22.0448,
                -18.09853,
                -10.00717,
                3.53743,
                7.556727,
                1.982921,
                0.2957763,
                5.842588,
                8.736755,
                -8.950642,
                -22.63761,
                -15.01393,
                -6.765015,
                -3.489682,
                -4.92079,
                -8.117239,
                -7.559782,
                2.183619,
                -1.424119,
                -7.351205,
                6.082211,
                -2.894077,
                -5.814854,
                8.810667,
                4.620723,
                6.450948,
                0.0002755493,
                0.6342525,
                5.568994,
                10.59651,
                0.8561473,
                -7.171011,
                -4.376842,
                -10.87645,
                -5.453482,
                1.69791,
                -3.412053,
                -3.004263,
                -8.861017,
                -9.14448,
                -7.718646,
                -4.476337,
                0.5089178,
                -0.02477878,
                3.158797,
                -5.017598,
                -9.551752,
                -6.084259,
                -14.41399,
                -19.64746,
                9.405186,
                21.59708,
                13.56969,
                25.48605,
                9.442548,
                7.706665,
                3.00517,
                -5.862418,
                13.00944,
                3.618364,
                7.934548,
                10.10879,
                6.392079,
                6.837857,
                -3.10954,
                3.921433,
                2.823433,
                0.3789104,
                0.8192734,
                0.570171,
                -2.732743,
                -1.753552,
                9.346987,
                11.0282,
                7.691156,
                1.892323,
                3.390249,
                3.787902,
                -9.100599,
                -10.29543,
                3.973159,
                2.22848,
                -0.1542701,
                7.706511,
                9.380507,
                -0.2247419,
                -14.08997,
                -16.70297,
                -16.25896,
                -1.425124
            ]

print(max(a))
print(min(a))

25.48605
-27.88097


In [40]:
from machine import Pin, ADC
from time import sleep

pot = ADC(Pin(33))
pot.atten(ADC.ATTN_11DB)       #Full range: 3.3v
pot.width(machine.ADC.WIDTH_12BIT)
while True:
  pot_value = pot.read()
  print(pot_value)
  sleep(1)

1959
1951
1950
1951
1942
1941
1936
1931
1929
1918
.1915
1908
1904
1897
1889
1888


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 11, in <module>
KeyboardInterrupt: 


In [46]:
print(runner.periph_manager.read_adc_buffer())
print(runner.output_buffer)

[1711, 2000, 2199, 2122, 1619, 1787, 2018, 2203, 1782, 1615, 1775, 1988, 2001, 1776, 1619, 1749, 2240, 2026, 1808, 1642, 2114, 2241, 2046, 1829, 1839, 2109, 2240, 2064, 1621, 1835, 2093, 1950, 1726, 1635, 1840, 2167, 1938, 1729, 1621, 2173, 2166, 1955, 1744, 1923, 2161, 2194, 1983, 1699, 1906, 2158, 2220, 1680, 1686, 1888, 2149, 1868, 1680, 1680, 2208, 2082, 1856, 1680, 2030, 2216, 2096, 1878, 1787, 2023, 2205, 2123, 1610, 1770, 1983, 2191, 1786, 1621, 1744, 1958, 2023, 1791, 1636, 1737, 2241, 2023, 1807, 1850, 2137, 2233, 2021, 1638, 1856, 2128, 2241, 1725, 1641, 1845, 2109, 1937, 1730, 1616, 1821, 2176, 1952, 1744, 1602, 2167, 2144, 1922, 1727, 1954, 2181, 2159, 1948, 1728, 1943, 2165, 2179, 1662, 1712, 1911, 2158, 1852, 1673, 1690, 1889, 2095, 1865, 1686, 1672, 2219, 2093, 1879, 1790, 2063, 2227, 2093, 1605, 1799, 2041, 2208, 1757, 1602, 1776, 1998, 1998, 1773, 1616, 1744, 2233, 2014, 1795, 1637, 2128, 2231, 2031, 1809, 1856, 2128, 2234, 1724, 1655, 1863, 2127, 1923, 1713, 1646, 184

In [10]:
%sendtofile lib/runner.py --source lib/runner.py



*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [44]:
%sendtofile main.py --source main.py

Sent 473 lines (8718 bytes) to main.py.


In [ ]:
%sendtofile lib/decodinga.py --source lib/decoding.py

In [ ]:
import network
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
wlan.connect('ssid', 'password')
print(wlan.ifconfig())

In [18]:
f = open("lib/config.py", "r")
print(f.read())

import utime

BASE_CLK_FREQ = 240000000  # 240 MHz for ESP32

ADC_SAMPLE_FREQ = 256  # sample freq in Hz

RECORDING_LEN_SEC = 4

OVERLAP = 0.8

DOWNSAMPLED_FREQ = 64  # 64 Hz downsampled  to ensure nyquist condition

PREPROCESSING = True  # if true, LP filter and downsample

STIM_FREQS = [7, 10, 12]  # stimulus freqs. in Hz

DEFAULT_LOG_SESSION = "test-{0}".format(utime.ticks_ms())

MODE = "log"

HTTP_LOG_URL = "http://james-tev.local:5000/"



## Using a Runner for experimentation and logging
The a `Runner` is encapsulates the core functions in this EEG system, including peripheral setup, sampling, signal processing, logging and memory management. The `OnlineRunner` offers mostly the same functionality as the standard `Runner` class, except it allows for logging and other communication with a remote server - either on the Internet or on your local network.

### Offline functionality
The standard `Runner` is good for testing core functionality without the need for remote logging. See below for initialisation and execution.

In [ ]:
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

### Calibration
If you are using an algorithm that leverages calibration data (MsetCCA, GCCA), you will need to record some calibration data to fit the decoder model. This is usually only done once off before inference starts. You may want to recalibrate at some semi-regular interval too though. 

At the moment, there is not an integrated process to record calibration data in the `Runner` class. You have to record calibration data and provide it to the runner which it will in turn use to fit its internal decoder model. In future, this will hopefully become more integrated and easy. For now, some random calibration data is generated below to illustrate the format which the runner/decoder expects. You need to provide iid calibration data trials for each stimulus frequency.

Note that if you try to run calibration using an incompatible algorithm (such as standard CCA), a warning will be generated and the calibration sequence will be skipped.

In [ ]:
gc.collect()

In [ ]:
from lib.synthetic import synth_X

calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
runner.calibrate(calibration_data)

In [ ]:
print(gc.mem_free())

### Decoding
When configured with a set of stimulus frequencies $\mathcal{F}=\{f_1, \dots, f_k, \dots, f_K\}$, the `Runner`'s decoder model consists of $K$ independent sub-classifiers $\Phi_k$ that each leverage the decoding algorithm selected. These independent classifiers must be calibrated independently. When the `Runner` is presented a new test observation, each sub-classifier $\Phi_k$ produces an output correlation estimate corresponding to $f_k$. Ultimately, the runner outputs a dictionary of frequency-correlation pairs of the form
```python
{f_1: 0.12, f_2: 0.03, f_3: 0.85}
```
The decoded output frequency is the one corresponding to the largest correlation in this output dictionary. In this example, it would be $f_3$.

In [ ]:
test_freq = 7 # 7 Hz test signal
test_data = synth_X(test_freq, Nc, Ns, Nt=1)

print(runner.decoder.classify(test_data))

In [16]:
from ulab import numpy as np
import utime as time
from lib.runner import Runner

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]
decoding_period_s = 4

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
#     print(data)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    
    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        trials.append(collectData(decode_period_s).flatten())
        print(trials)
    runner.stop()

    print(len(trials))

    gc.collect()
    return np.array(trials)


ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


In [17]:
#calibration_data = {f:getData(Nt, decoding_period_s) for f in stim_freqs}
calibration_data = {}
calibration_data[7] = getData(Nt, decoding_period_s)
gc.collect()

..[array([0.1022726, 6.393758, 30.04326, ..., -17.88625, 40.25251, 43.99335], dtype=float32)]
[array([0.1022726, 6.393758, 30.04326, ..., -17.88625, 40.25251, 43.99335], dtype=float32), array([0.02554362, 4.924626, 34.40299, ..., 13.66638, -37.46715, -46.48108], dtype=float32)]
.[array([0.1022726, 6.393758, 30.04326, ..., -17.88625, 40.25251, 43.99335], dtype=float32), array([0.02554362, 4.924626, 34.40299, ..., 13.66638, -37.46715, -46.48108], dtype=float32), array([0.04832244, 5.792087, 36.82669, ..., 47.1671, 5.583657, -50.93529], dtype=float32)]
3


In [18]:
calibration_data[10] = getData(Nt, decoding_period_s)
gc.collect()

..[array([0.08628019, 6.923466, 38.24591, ..., -18.19421, 34.08795, 46.6745], dtype=float32)]
[array([0.08628019, 6.923466, 38.24591, ..., -18.19421, 34.08795, 46.6745], dtype=float32), array([0.05372761, 5.886846, 36.22691, ..., 20.57845, -30.37518, -49.47893], dtype=float32)]
.[array([0.08628019, 6.923466, 38.24591, ..., -18.19421, 34.08795, 46.6745], dtype=float32), array([0.05372761, 5.886846, 36.22691, ..., 20.57845, -30.37518, -49.47893], dtype=float32), array([0.1091523, 5.572274, 22.46525, ..., 39.65464, 51.01727, 7.14376], dtype=float32)]
3


In [19]:
calibration_data[12] = getData(Nt, decoding_period_s)
gc.collect()

..[array([-0.07902402, -2.427289, -2.418796, ..., -60.14733, -24.28555, 30.13657], dtype=float32)]
.[array([-0.07902402, -2.427289, -2.418796, ..., -60.14733, -24.28555, 30.13657], dtype=float32), array([-0.05558455, -0.1863209, 11.00656, ..., 39.97823, 51.31582, 2.343739], dtype=float32)]
[array([-0.07902402, -2.427289, -2.418796, ..., -60.14733, -24.28555, 30.13657], dtype=float32), array([-0.05558455, -0.1863209, 11.00656, ..., 39.97823, 51.31582, 2.343739], dtype=float32), array([-0.04097127, 0.7651208, 15.67703, ..., -21.87494, 36.72865, 55.21906], dtype=float32)]
3


In [20]:
print(gc.mem_free())
del runner
print(gc.mem_free())

44400
44400


In [21]:
decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)
print(gc.mem_free())

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
45360


In [22]:
decode.run()

In [26]:
print(decode.decoder.classify(np.array(decode.output_buffer)))

.

*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [27]:
decode.stop()

.

*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [28]:
print(calibration_data)

.

*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [29]:
print(1)

1


### Asynchronous operation
Once the `Runner` has been configured and calibrated (if applicable), its internal `run()` loop can be started in which it will asynchronously sample and decode EEG data at preconfigured frequencies. Timing is handled using hardware timers on the ESP32 and interrupts are used to run asynchronous ISRs that handle sampling, preprocessing, filtering and decoding.

Note that once the async run loop has begun, you can still run commands or view the `Runner`'s attributes although there may be a noticeable delay since ISRs will typically get higher execution priority and there are quite a few interrupt loops running.

In [ ]:
# start sampling and recording data (logging not setup in this case)
runner.run()

In [ ]:
# see if runner has indeed started smapling
print(runner.is_sampling)

In [ ]:
# display the contents of the output buffer - this will be updated internally by the runner
# at a rate determined by the sampling frequency and sample buffer size (typically every 1s)
print(runner.output_buffer)

In [ ]:
# decode the contents of the output buffer. There will be a delay here if the runner 
# is currently running (i.e. `is_sampling=True`).
print(runner.decode())

In [ ]:
# stop runner
runner.stop()

#### Simple decoding loop
In order to test online decoding, here is a basic synchronous loop-based option. Interrupt the cell to stop the infinite loop.

In [ ]:
import utime as time
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

decode_period_s = 2 # read decoded output every x seconds

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup()

if decoding_algo in ['MsetCCA', 'GCCA']:
    from lib.synthetic import synth_X

    calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
    runner.calibrate(calibration_data)

runner.run() # start async run loop

try:
    while True:
        time.sleep(decode_period_s)
        print(runner.decoded_output)
except KeyboardInterrupt:
    runner.stop()
    print('received SIGINT - stopping')

### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [ ]:
%sendtofile --source /Users/rishil/Desktop/.env lib/.env  --binary

In [34]:
from lib.utils import connect_wifi, load_env_vars
from lib.synthetic import synth_X

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'
calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


#### Online Runner
Now that you've established network connectivitiy, you can test out an `OnlineRunner`. In order to test web logging to a remote server, we can use a basic HTTP logger. However, this obviously needs an API/server willing to accept our requests. There is a basic logging API using `Flask` in `/eeg_lib/logging_server.py`. You can run it using `python logging_server.py` which will spin up a development server on the predefined port (5000 or 5001). Then, just configure your `OnlineRunner` with the appropriate logger params and you're set.

In [35]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://192.168.0.13:5001/" # make sure the port corresponds to your logging server configuration
log_params = dict(server=api_host, log_period=4, logger_type=logger_types.HTTP, send_raw=True, session_id='test')
# runner = OnlineRunner('CCA', buffer_size=256)
runner = OnlineRunner(decoding_algo, buffer_size=Ns)
runner.setup(**log_params)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [36]:
# start the runner - you should see requests being made to your local server
runner.calibrate(calibration_data)
runner.run()

In [37]:
runner.stop()

## Experimentation

In [ ]:
%rebootdevice

In [ ]:
%lsmagic